**Title:** Mainpipe Data Preparation Pipeline<br>
**Author:** Mary Kolyaei<br>
**Email:** maryamkolyaie@gmail.com<br>
**Sources:** <br>
AWS: https://aws.amazon.com/blogs/machine-learning/an-introduction-to-preparing-your-own-dataset-for-llm-training/<br>
Hugging Face: https://huggingface.co/docs/datasets/en/quickstart<br>
https://huggingface.co/docs/datasets/v1.4.0/loading_datasets.html


# Pkgs

In [1]:
from pkg import install_packages
install_packages()

Installing pandas...
pandas installed successfully.

Installing numpy...
numpy installed successfully.

Installing matplotlib...
matplotlib installed successfully.

Installing langdetect...
langdetect installed successfully.

Installing fastparquet...
fastparquet installed successfully.

Installing lxml...
lxml installed successfully.

Installing transformers...
transformers installed successfully.

Installing detoxify torch --quiet...
Failed to install detoxify torch --quiet: Command '['pip', 'install', '-U', 'detoxify torch --quiet']' returned non-zero exit status 1.



# Libraries

In [1]:
import logging       # logging framework
logger = logging.getLogger("stage8_sharding")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(name)s - %(message)s"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

!python run_pipeline.py #executes every stage of the end-to-end data pipeline

🌱 Stage 1 — Ingestion

**Script:** `ingest.py`  
**Input:** `mainpipe_data_v1.jsonl`  
**Output:** `mainpipe_ingested_v1.parquet`

### Steps
- Loads raw `.jsonl` dataset  
- Creates stable `doc_id` using SHA1 hash  
- Adds ingestion timestamp (`ingest_ts`)  
- Adds metadata fields (`source`)  
- Saves an ingested, structured Parquet file


🧹 Stage 2 — Text Cleaning & Filtering

**Script:** text_clean_and_filter.py
**Input:** mainpipe_ingested_v1.parquet
**Outputs:** mainpipe_cleaned_v2.parquet  &  mainpipe_cleaned_v2.jsonl  &  mainpipe_dropped_v2.parquet

### Steps
- Removes empty, whitespace & extremely short texts
- Normalises whitespace & punctuation
- Computes basic content stats
- Basic filtering (empty, too short, etc.)


🧽 Stage 3 — Deep Cleaning + PII Masking

**Script:** deep_clean_and_pii.py
**Input:** mainpipe_cleaned_v2.parquet
**Outputs:** mainpipe_cleaned_v4.parquet  &  mainpipe_dropped_v4.parquet  &  mainpipe_cleaned_v4.jsonl

### Steps
- Removes HTML tags
- Removes boilerplate (cookie banners, footers, disclaimers)
- Normalises repeated characters
- Detects and masks: <EMAIL>, <PHONE>, <CREDIT_CARD>, <IBAN>
- Applies token-based heuristics:
- stopword ratio
- unique-token ratio
- repetitive-token spam
- Drops low-information documents


🔍 Stage 4 — Deduplication (Exact + Near-Dup)

**Script:** duplication.py
**Input:** mainpipe_cleaned_v4.parquet
**Outputs:** mainpipe_cleaned_v5.parquet  &  mainpipe_dropped_v5.parquet  &  mainpipe_cleaned_v5.jsonl

### Steps
- Builds canonical version of each text
- Exact dedup: SHA256 hash
- Near-dup: match first 500 chars of canonical text
- Drops duplicate documents, drop_reason, etc


⭐ Stage 5 — Scoring & Mixture Assignment

**Script:** scoring_and_mixture.py
**Input:** mainpipe_cleaned_v5.parquet
**Outputs:**  mainpipe_scored_v6.parquet  &  mainpipe_scored_v6.jsonl

### Steps
- Computes quality_score ∈ [0, 1] using:
- language confidence
- token count
- unique-token ratio
- PII penalty


🧩 Stage 6 — Tokenisation + Training JSONL Export

**Script:** Tokenisation_JSONL_export.py
**Input:** mainpipe_scored_v6.parquet
**Outputs:** mainpipe_tokenised_v7.parquet  &  train_web_sample_tokenised.jsonl

### Steps
- Loads HuggingFace tokenizer (GPT-2)
- input_ids
- attention_mask
- n_tokens
- Drops extremely short or extremely long docs
- Saves training-ready JSONL for model training


📦 Stage 7 — Sharding

**Script:** sharding.py
**Input:** mainpipe_tokenised_v7.parquet
**Outputs:** Shard files (e.g., train_shard_00001.jsonl, train_shard_00002.jsonl, …)  &  manifest.json  &  tiny_train.jsonl

### Steps
- Splits tokenised dataset into smaller .jsonl chunks
- Writes a manifest describing:
- number of shards
- number of docs
- total tokens
- tokenizer name
- file paths

In [1]:
#executes every stage of the end-to-end data pipeline
!python run_pipeline.py 

2025-11-27 18:21:01,060 - INFO - ==== Running full data pipeline ====
2025-11-27 18:21:01,061 - INFO - Project root: C:\Users\mkolyaei\Desktop\Maincode
2025-11-27 18:21:01,062 - INFO - ================================================================================
2025-11-27 18:21:01,062 - INFO - Starting Stage 1: ingest:Raw JSONL → Ingested Parquet pipeline
2025-11-27 18:21:01,063 - INFO - Command: python ingest.py
2025-11-27 18:21:15,265 - INFO - ✅ Stage 1: ingest:Raw JSONL → Ingested Parquet pipeline completed successfully
2025-11-27 18:21:15,265 - INFO - ================================================================================
2025-11-27 18:21:15,265 - INFO - Starting Stage 2: text_clean_and_filter
2025-11-27 18:21:15,270 - INFO - Command: python text_clean_and_filter.py
2025-11-27 19:56:48,658 - INFO - ✅ Stage 2: text_clean_and_filter completed successfully
2025-11-27 19:56:48,660 - INFO - ================================================================================
202

Loading Stage 6 scored parquet from: mainpipe_scored_v6.parquet
Loading tokenizer: gpt2

Token count stats (n_tokens):
            n_tokens
count  191507.000000
mean      318.047909
std       499.923922
min         0.000000
25%        85.000000
50%       178.000000
75%       358.000000
max     16365.000000
=== Stage 7: Token length filter summary ===
Input rows           : 191507
Kept rows            : 187855
Dropped rows         : 3652

Drop reasons (incl. previous stages if any):
drop_reason
too_many_tokens    2614
too_few_tokens     1038
Name: count, dtype: int64

>>> Final training docs: 187855
>>> Dropped (too short/long or earlier reasons): 3652

Writing tokenised parquet to: mainpipe_tokenised_v7.parquet
Writing JSONL to train_web_sample_tokenised.jsonl ...
Done.

Sample lines from tokenised JSONL:
{"input_ids": [818, 262, 1239, 7464, 3344, 284, 5755, 12926, 286, 663, 36814, 7557, 11, 6283, 3996, 23299, 1666, 422, 262, 12926, 6355, 2055, 11, 257, 5863, 8489, 290, 2766, 422, 1111

2025-11-27 20:06:38,314 - INFO - ==== Running full data pipeline ====
2025-11-27 20:06:38,315 - INFO - Project root: C:\Users\mkolyaei\Desktop\Maincode
2025-11-27 20:06:38,315 - INFO - ================================================================================
2025-11-27 20:06:38,315 - INFO - Starting Stage 1: ingest:Raw JSONL \u2192 Ingested Parquet pipeline
2025-11-27 20:06:38,315 - INFO - Command: python ingest.py
2025-11-27 20:06:53,201 - INFO - \u2705 Stage 1: ingest:Raw JSONL \u2192 Ingested Parquet pipeline completed successfully
2025-11-27 20:06:53,201 - INFO - ================================================================================
2025-11-27 20:06:53,201 - INFO - Starting Stage 2: text_clean_and_filter
2025-11-27 20:06:53,201 - INFO - Command: python text_clean_and_filter.py
2025-11-27 21:41:20,207 - INFO - \u2705 Stage 2: text_clean_and_filter completed successfully
2025-11-27 21:41:20,208 - INFO - ================================================================

In [3]:
!python plots_charts.py

2025-11-27 22:27:24,567 - INFO - Reading mainpipe_cleaned_v2.parquet
2025-11-27 22:27:26,909 - INFO - Reading mainpipe_cleaned_v4.parquet
2025-11-27 22:27:30,600 - INFO - Reading mainpipe_scored_v6.parquet
2025-11-27 22:27:34,659 - INFO - Reading mainpipe_tokenised_v7.parquet
2025-11-27 22:27:39,368 - INFO - Reading mainpipe_dropped_v2.parquet
2025-11-27 22:27:39,630 - INFO - Reading mainpipe_dropped_v4.parquet
2025-11-27 22:27:39,637 - INFO - Reading mainpipe_dropped_v5.parquet
2025-11-27 22:27:40,318 - INFO - Saved plot: plots\hist_n_tokens.png
2025-11-27 22:27:40,511 - INFO - Saved plot: plots\hist_char_len.png
2025-11-27 22:27:40,691 - INFO - Saved plot: plots\hist_word_count.png
2025-11-27 22:27:40,845 - INFO - Saved plot: plots\hist_lang_score.png
2025-11-27 22:27:40,993 - INFO - Saved plot: plots\hist_quality_score.png
2025-11-27 22:27:41,149 - INFO - Saved plot: plots\bar_drop_reasons.png
2025-11-27 22:27:41,304 - INFO - Saved plot: plots\hist_alpha_ratio.png
2025-11-27 22:27:4